# Accuracy Metrics

Goal here is to develop three different accuracy metrics for the EEG online decoder. The three accuracy metrics are described as follows:

**1. Fraction of time target is acquired:** The decoder will move in the direction of the on screen targets. Eventually the decoer will saturate and be at the same location as the on-screen target. This method will compute accuracy as the fraction of time the decoders position is equal to the target position (divided by the fraction of time the decoder is not at the center of the screen). The decoder is assumed to be inactive when it is at the center of the screen so these timesteps are omitted from the accuracy calculations

**2. Fraction of time decoder is in the correct direction:** This computes the fraction of timesteps the decoder is closest to the correct target.

**3. Fraction of time decoder has the correct velocity:** This computes the velocity of the decoder at every timestep. The fraction of these timesteps where the velocity is in the direction of the correct target (relative to all active timesteps) is then reported as accuracy.

In [1]:
# imports

import sys
# setting path
sys.path.append('/home2/brandon/Offline_EEGNet/')


import numpy as np
import matplotlib.pyplot as plt
import preprocessing as pre
from preprocessing import load_data, parse_labels, partition_data, augment_data
from train import train
from torch.utils.data import TensorDataset, DataLoader
import torch
import pdb
from EEGNet import EEGNet
import argparse
import tqdm
from collections import defaultdict
import data.data_util as util

# Accuracy Metrics for 2D Decoding 

In [8]:
# load in the data
data_file_name = "SJ_CL_Run2_CL_DDPG_Replay"
fpath = "/home2/brandon/Offline_EEGNet/data/" + data_file_name
eeg_data = util.load_data(fpath + "/eeg.bin")
task_data = util.load_data(fpath + "/task.bin")

In [9]:
# partition the data into trials
# extract trial start times
new_state_bool = np.zeros(task_data['state_task'].size, dtype='bool')
new_state_bool[1:] = (task_data['state_task'][1:] != task_data['state_task'][:-1])
new_state_inds = np.nonzero(new_state_bool)[0]
trial_labels = task_data['state_task'][new_state_inds]
trial_start_times = task_data['time_ns'][new_state_inds] 
eeg_timing = eeg_data['time_ns']




eeg_trials = defaultdict(lambda: [])  
# partition trials
for trial_ix in range(len(trial_labels)-1):
    if int(trial_labels[trial_ix]) == 4:  # skip rest trials
        continue 

    # extract eeg_data indices were trial started and ended
    trial_start_ix = np.where(task_data["time_ns"] > trial_start_times[trial_ix])[0][0] + 50 # omit first second of data
    trial_end_ix = np.where(task_data["time_ns"] < trial_start_times[trial_ix+1])[0][-1]
    
    # get the decoder position
    eeg_trials[trial_labels[trial_ix]].append(task_data["decoded_pos"][trial_end_ix])

    # append data from 64 electrodes
    #eeg_trials.append(eeg_data['databuffer'][trial_start_ix:trial_end_ix, :64])
    #min_trial_len = np.minimum(eeg_trials[-1].shape[0], min_trial_len).astype(int)
    #eeg_trial_labels.append(class_names[trial_labels[trial_ix]])

In [10]:
targets = np.unique(task_data["state_task"]) 
targets = list(filter(lambda x: x != 4, targets))  # remove resting state
n_targets = len(targets)
print("number of targets:", n_targets)
print("targets:", targets)

number of targets: 4
targets: [0, 1, 2, 3]


## Compute Weak Accuracy Metric for 2D Decoding

The accuracy is computed for the 2D decoding task (left/right/up/down). A trial is considered accurate as long as the decoded cursor position ends on the correct half of the screen. 

In [16]:
# maps target label to decoder 2D (x,y) position
target2pos = {
    0 : [-1, 0],  # left
    1 : [1, 0],   # right
    2 : [0, -1],  # down
    3 : [0, 1]    # up
}

true_targets = []
predicted_targets = []

# maps each target to a list of bools indicating if that trial is correct
correct_trials = defaultdict(lambda: 0)  
n_trials = defaultdict(lambda: 0)  # number of trials for each condition

# list of decoder positions at the end of all trials
# e.g. final_decoder_pos[0] is a list of (x,y)-coordinates 
# of the final decoder position at the end of each trial 
# associated with left (target=0) reaches
final_decoder_pos = defaultdict(lambda: 0)
decoder_acc = defaultdict(lambda: 0)
for target in targets:
    for i in range(len(eeg_trials[target])):
        true_targets.append(target)
    # eeg_trials[target] is a list with length equal to the number of target trials.
    # Each array is a timeseries of 2D position of the decoder for that trial
    print("eeg_trials:", np.array(eeg_trials[target]).shape)
    final_decoder_pos[target] = np.array(np.sign(eeg_trials[target]))  # final x,y-position of decoder on each trial (n_trials, 2) 
    print("final decoder pos:", final_decoder_pos[target].shape)
    
    # get predited target
    for i in range(len(final_decoder_pos[target])):
        if final_decoder_pos[target][i][0] == target2pos[0][0] and final_decoder_pos[target][i][1] == target2pos[0][1]:
            predicted_targets.append(0)
        elif final_decoder_pos[target][i][0] == target2pos[1][0] and final_decoder_pos[target][i][1] == target2pos[1][1]:
            predicted_targets.append(1)
        elif final_decoder_pos[target][i][0] == target2pos[2][0] and final_decoder_pos[target][i][1] == target2pos[2][1]:
            predicted_targets.append(2)
        elif final_decoder_pos[target][i][0] == target2pos[3][0] and final_decoder_pos[target][i][1] == target2pos[3][1]:
            predicted_targets.append(3)
        else:
            predicted_targets.append(4)
    
    if target < 2: # L/R
        correct_trials[target] = final_decoder_pos[target][:,0] == target2pos[target][0]  # signs are the same when it is closest
    else: # U/D
        correct_trials[target] = final_decoder_pos[target][:,1] == target2pos[target][1]  # signs are the same when it is closest
    correct_trials[target] = np.sum(correct_trials[target])
    #correct_steps = np.where(correct_steps==2)[0].shape[0]
    
    # total number of this type of trial
    n_trials[target] = final_decoder_pos[target].shape[0] 
    
    # accuracy on this trial type
    acc = correct_trials[target]*100./n_trials[target]
    decoder_acc[target] = np.maximum(0., acc)  # negative accuracy indicates unknown class
    if True:
        print("\nTarget:", target)
        print("correct trials:", correct_trials[target])
        print("trials of this type:", n_trials[target])
        print("accuracy(%):", acc )

total_correct_trials = 0
total_trials = 0
for target in targets:
    total_correct_trials += correct_trials[target]
    total_trials += n_trials[target]
acc = total_correct_trials * 100. / total_trials
print("\n\nAverage accuracy across all trials(%):", acc)
    
# decoder_acc[target_ix] = accuracy_of_decoder_for_that_target

eeg_trials: (7, 2)
final decoder pos: (7, 2)

Target: 0
correct trials: 0
trials of this type: 7
accuracy(%): 0.0
eeg_trials: (5, 2)
final decoder pos: (5, 2)

Target: 1
correct trials: 2
trials of this type: 5
accuracy(%): 40.0
eeg_trials: (5, 2)
final decoder pos: (5, 2)

Target: 2
correct trials: 0
trials of this type: 5
accuracy(%): 0.0
eeg_trials: (4, 2)
final decoder pos: (4, 2)

Target: 3
correct trials: 1
trials of this type: 4
accuracy(%): 25.0


Average accuracy across all trials(%): 14.285714285714286


## Compute Strict Accuracy Metric
Decoder is accurate if it ends within a threshold position of target

## Compute accuracy metrics

In [14]:
predicted_targets

[0,
 0,
 4,
 4,
 4,
 0,
 4,
 0,
 0,
 3,
 3,
 4,
 4,
 0,
 3,
 0,
 0,
 4,
 0,
 4,
 0,
 0,
 0,
 4,
 0,
 3,
 4,
 3,
 4,
 0,
 0,
 4,
 4]

In [18]:
import sklearn.metrics
plt.imshow(sklearn.metrics.confusion_matrix(true_targets, predicted_targets))
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")

Text(119.04722222222226, 0.5, 'True Labels')

In [11]:
def distance_from_target(final_decoder_pos, target_pos):
    '''returns distance of decoder from the target
    
    parameters
    -final_decoder_pos: NumPy array of the final decoder position in 2D. Has shape
        (n_trials, 2)
    -target_pos: NumPy array or list of the target position in 2D. If numpy array 
        has shape (2,) or if list has length 2
    '''
    
    x_dist_square = (final_decoder_pos[:,0] - target_pos[0])**2
    y_dist_square = (final_decoder_pos[:,1] - target_pos[1])**2
    
    dist = np.sqrt(x_dist_square + y_dist_square)

    return dist   # distance from target at the end of each trial (n_trials,)

In [12]:
# maps target label to decoder 2D (x,y) position
target2pos = {
    0 : [-75, 0],  # left
    1 : [75, 0],   # right
    2 : [0, -75],  # down
    3 : [0, 75]    # up
}

# how close decoder must be to target to be correct
target_thresh = 30   # 2/3 of the way to target

# maps each target to a list of bools indicating if that trial is correct
correct_trials = defaultdict(lambda: 0)  
n_trials = defaultdict(lambda: 0)  # number of trials for each condition

# list of decoder positions at the end of all trials
# e.g. final_decoder_pos[0] is a list of (x,y)-coordinates 
# of the final decoder position at the end of each trial 
# associated with left (target=0) reaches
final_decoder_pos = defaultdict(lambda: 0)
decoder_acc = defaultdict(lambda: 0)
for target in targets:
    # eeg_trials[target] is a list with length equal to the number of target trials.
    # Each array is a timeseries of 2D position of the decoder for that trial
    final_decoder_pos[target] = np.array(eeg_trials[target])  # final x,y-position of decoder on each trial (n_trials, 2) 
    
    # get distnace from target at the end of each trial
    distances = distance_from_target(final_decoder_pos[target], target2pos[target])
    # sum the number of trials that ended close to the target
    correct_trials[target] = distances < target_thresh
    correct_trials[target] = np.sum(correct_trials[target])
    
    # total number of this type of trial
    n_trials[target] = final_decoder_pos[target].shape[0] 
    
    # accuracy on this trial type
    acc = correct_trials[target]*100./n_trials[target]
    decoder_acc[target] = np.maximum(0., acc)  # negative accuracy indicates unknown class
    if True:
        print("\nTarget:", target)
        print("correct trials:", correct_trials[target])
        print("trials of this type:", n_trials[target])
        print("accuracy(%):", acc )

total_correct_trials = 0
total_trials = 0
for target in targets:
    total_correct_trials += correct_trials[target]
    total_trials += n_trials[target]
acc = total_correct_trials * 100. / total_trials
print("\n\nAverage accuracy across all trials(%):", acc)
    
# decoder_acc[target_ix] = accuracy_of_decoder_for_that_target


Target: 0
correct trials: 14
trials of this type: 24
accuracy(%): 58.333333333333336

Target: 1
correct trials: 21
trials of this type: 30
accuracy(%): 70.0

Target: 2
correct trials: 10
trials of this type: 20
accuracy(%): 50.0

Target: 3
correct trials: 25
trials of this type: 26
accuracy(%): 96.15384615384616


Average accuracy across all trials(%): 70.0


## Compute strict accuracy for center out task

In [32]:
# load in the data
data_file_name = "BM_co_movement_0"
fpath = "C:/Users/DNNeu/KaoLab/EEG_Decoding/Offline_EEG_Decoding/data/" + data_file_name
eeg_data = util.load_data(fpath + "/eeg.bin")
task_data = util.load_data(fpath + "/task.bin")

# partition the data into trials
# extract trial start times
new_state_bool = np.zeros(task_data['state_task'].size, dtype='bool')
new_state_bool[1:] = (task_data['state_task'][1:] != task_data['state_task'][:-1])
new_state_inds = np.nonzero(new_state_bool)[0]
trial_labels = task_data['state_task'][new_state_inds]
trial_start_times = task_data['time_ns'][new_state_inds] 
eeg_timing = eeg_data['time_ns']




eeg_trials = defaultdict(lambda: [])  
# partition trials
for trial_ix in range(len(trial_labels)-1):
    if int(trial_labels[trial_ix]) == 8:  # skip rest trials
        continue 

    # extract eeg_data indices were trial started and ended
    trial_start_ix = np.where(task_data["time_ns"] > trial_start_times[trial_ix])[0][0] + 50 # omit first second of data
    trial_end_ix = np.where(task_data["time_ns"] < trial_start_times[trial_ix+1])[0][-1]
    
    # get the decoder position
    eeg_trials[trial_labels[trial_ix]].append(task_data["decoded_pos"][trial_end_ix])

    # append data from 64 electrodes
    #eeg_trials.append(eeg_data['databuffer'][trial_start_ix:trial_end_ix, :64])
    #min_trial_len = np.minimum(eeg_trials[-1].shape[0], min_trial_len).astype(int)
    #eeg_trial_labels.append(class_names[trial_labels[trial_ix]])
    
targets = np.unique(task_data["state_task"]) 
targets = list(filter(lambda x: x != 8, targets))  # remove resting state
n_targets = len(targets)
print("number of targets:", n_targets)
print("targets:", targets)

number of targets: 7
targets: [0, 1, 2, 3, 4, 5, 6]


In [33]:
# maps target label to decoder 2D (x,y) position
'''target2pos = {
    0 : [-249, 0],  # left
    1 : [240, 0],   # right
    2 : [0, -240],  # down
    3 : [0, 240]    # up
}'''


target2pos = {
  0 : [240, 0],
  1 : [169, 169],
  2 : [0, 240],
  3 : [-169, 169],
  4 : [-249, 0],
  5 : [-169, -169],
  6 : [0, -240],
  7 : [169, -169]
}


# how close decoder must be to target to be correct
target_thresh = 30   # 2/3 of the way to target

# maps each target to a list of bools indicating if that trial is correct
correct_trials = defaultdict(lambda: 0)  
n_trials = defaultdict(lambda: 0)  # number of trials for each condition

# list of decoder positions at the end of all trials
# e.g. final_decoder_pos[0] is a list of (x,y)-coordinates 
# of the final decoder position at the end of each trial 
# associated with left (target=0) reaches
final_decoder_pos = defaultdict(lambda: 0)
decoder_acc = defaultdict(lambda: 0)
for target in targets:
    # eeg_trials[target] is a list with length equal to the number of target trials.
    # Each array is a timeseries of 2D position of the decoder for that trial
    final_decoder_pos[target] = np.array(eeg_trials[target])  # final x,y-position of decoder on each trial (n_trials, 2) 
    
    # get distnace from target at the end of each trial
    distances = distance_from_target(final_decoder_pos[target], target2pos[target])
    # sum the number of trials that ended close to the target
    correct_trials[target] = distances < target_thresh
    correct_trials[target] = np.sum(correct_trials[target])
    
    # total number of this type of trial
    n_trials[target] = final_decoder_pos[target].shape[0] 
    
    # accuracy on this trial type
    acc = correct_trials[target]*100./n_trials[target]
    decoder_acc[target] = np.maximum(0., acc)  # negative accuracy indicates unknown class
    if True:
        print("\nTarget:", target)
        print("correct trials:", correct_trials[target])
        print("trials of this type:", n_trials[target])
        print("accuracy(%):", acc )

total_correct_trials = 0
total_trials = 0
for target in targets:
    total_correct_trials += correct_trials[target]
    total_trials += n_trials[target]
acc = total_correct_trials * 100. / total_trials
print("\n\nAverage accuracy across all trials(%):", acc)
    
# decoder_acc[target_ix] = accuracy_of_decoder_for_that_target


Target: 0
correct trials: 3
trials of this type: 3
accuracy(%): 100.0

Target: 1
correct trials: 3
trials of this type: 3
accuracy(%): 100.0

Target: 2
correct trials: 6
trials of this type: 6
accuracy(%): 100.0

Target: 3
correct trials: 1
trials of this type: 1
accuracy(%): 100.0

Target: 4
correct trials: 2
trials of this type: 7
accuracy(%): 28.571428571428573

Target: 5
correct trials: 1
trials of this type: 4
accuracy(%): 25.0

Target: 6
correct trials: 3
trials of this type: 6
accuracy(%): 50.0


Average accuracy across all trials(%): 63.333333333333336
